<a href="https://colab.research.google.com/github/pAppr69/Win-Factor_yashjeet/blob/main/submissions_Yashjeet/basic_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
url = 'https://media.githubusercontent.com/media/pAppr69/Win-Factor_yashjeet/refs/heads/main/mw_pw_profiles.csv'
df = pd.read_csv(url)

<ipython-input-5-1745788563>:2: DtypeWarning: Columns (32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [10]:
def get_dataframe_info(df):

    info_dict = {
        'Column_Name': df.columns,
        'Data_Type': df.dtypes,
        'NaN_Percentage': (df.isna().sum() / len(df) * 100).round(2),
    }

    print(pd.DataFrame(info_dict))
get_dataframe_info(df)

                                 Column_Name Data_Type  NaN_Percentage
player_id                          player_id    object            0.00
match_id                            match_id    object            0.00
gender                                gender    object            0.00
balls_per_over                balls_per_over     int64            0.00
start_date                        start_date    object            0.00
series_name                      series_name    object            0.47
match_type                        match_type    object            0.00
name_x                                name_x    object            0.00
runs_scored                      runs_scored     int64            0.00
player_out                        player_out     int64            0.00
balls_faced                      balls_faced     int64            0.00
fours_scored                    fours_scored     int64            0.00
sixes_scored                    sixes_scored     int64            0.00
catche

In [ ]:
df = df[['player_id','start_date', 'catches_taken', 'run_out_direct', 'run_out_throw', 'stumpings_done', 'balls_bowled',
              'runs_conceded', 'wickets_taken', 'maidens',
              'runs_scored', 'balls_faced', 'fours_scored', 'sixes_scored', 'fantasy_score_total']]
df

,player_id,start_date,catches_taken,run_out_direct,run_out_throw,stumpings_done,balls_bowled,runs_conceded,wickets_taken,maidens,runs_scored,balls_faced,fours_scored,sixes_scored,fantasy_score_total
0,15688,21-08-18,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,30.0,1.0,0.0,17.0
1,15688,05-05-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,18.0,1.0,0.0,7.0
2,15688,09-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,5.0
3,15688,15-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
4,15688,14-09-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90034,399b0b94,03-06-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,103.0,10.0,1.0,88.0
90035,399b0b94,04-07-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0,103.0,4.0,0.0,46.0
90036,399b0b94,11-07-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,51.0,3.0,0.0,33.0
90037,399b0b94,06-03-21,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,84.0,2.0,0.0,31.0


In [ ]:
def calculate_ema_stats(df, span=14):
    df = df.copy()
    df['start_date'] = pd.to_datetime(df['start_date'])
    df = df.sort_values(['player_id', 'start_date'])


    stat_columns = [col for col in df.columns if col not in ['player_id', 'start_date']]

    result = df[['player_id']].copy()

    for col in stat_columns:
        result[f'{col}_ema'] = df.groupby('player_id')[col].transform(
            lambda x: x.ewm(span=span, adjust=False).mean()
        )

    result = result.loc[df.groupby('player_id')['start_date'].idxmax()]

    final_columns = ['player_id'] + [col for col in result.columns if col.endswith('_ema')]
    result = result[final_columns]

    return result
df = calculate_ema_stats(df)
df

<ipython-input-9-2099705641>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_date'] = pd.to_datetime(df['start_date'])


,player_id,catches_taken_ema,run_out_direct_ema,run_out_throw_ema,stumpings_done_ema,balls_bowled_ema,runs_conceded_ema,wickets_taken_ema,maidens_ema,runs_scored_ema,balls_faced_ema,fours_scored_ema,sixes_scored_ema,fantasy_score_total_ema
11,00029c30,0.000000,0.0000,0.000000,0.000000,11.726242,14.373774,0.730086,0.000000,0.491496,0.997823,0.000000,0.000000,24.349301
12,0019ca13,0.000000,0.0000,0.000000,0.000000,6.000000,8.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,1.000000
32,0030a57d,0.568483,0.0000,0.068655,0.329083,0.000000,0.000000,0.000000,0.000000,0.692458,2.226110,0.000000,0.000000,12.529592
64,00321fff,0.076681,0.0000,0.000000,0.000000,16.239733,18.829289,0.720689,0.029666,1.340845,1.937817,0.121957,0.000000,28.588857
132,003d49e6,0.423060,0.0000,0.002425,0.000000,11.342682,15.596999,0.205820,0.200425,8.512050,6.577126,0.701626,0.387214,23.006045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9844,7160156,0.100148,0.0000,0.000000,0.000000,1.340890,2.117115,0.231111,0.000000,1.364168,6.451844,0.133333,0.000000,12.467575
379,784913,0.454047,0.0565,0.000000,0.000000,35.231559,28.918509,1.394232,0.056500,2.392818,4.286695,0.265489,0.065193,46.324317
10830,8320616,0.000000,0.0000,0.000000,0.000000,17.493333,19.226074,0.115556,0.000000,1.466667,5.552889,0.000000,0.000000,6.496296
666,863408,0.965501,0.0000,0.048967,0.098938,10.352584,11.283026,0.532915,0.000000,17.507441,22.380575,1.577763,0.577250,49.368981


In [ ]:
df.to_csv('/data.csv')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import shap

In [ ]:
def train_random_forest(df, target_col, feature_cols=None, n_estimators=100, random_state=42):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', target_col]]

    X = df[feature_cols]
    y = df[target_col]

    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    model.fit(X, y)

    predictions = model.predict(X)

    mse = mean_squared_error(y, predictions)
    r2 = r2_score(y, predictions)

    print("Random Forest Regressor Results:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

    importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance:")
    print(importance)

    output_df = df[['player_id',  target_col]].copy()
    output_df['predicted_value'] = predictions

    return model, output_df, importance



In [ ]:
def train_xgboost(df, target_col, feature_cols=None, n_estimators=100, random_state=42):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', target_col]]

    X = df[feature_cols]
    y = df[target_col]

    model = xgb.XGBRegressor(n_estimators=n_estimators, random_state=random_state)
    model.fit(X, y)

    predictions = model.predict(X)

    mse = mean_squared_error(y, predictions)
    r2 = r2_score(y, predictions)

    print("XGBoost Regressor Results:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"R² Score: {r2:.4f}")

    importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("\nFeature Importance:")
    print(importance)

    output_df = df[['player_id',  target_col]].copy()
    output_df['predicted_value'] = predictions

    return model, output_df, importance


In [ ]:
def apply_pca(df, feature_cols=None, n_components=2):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', ]]

    X = df[feature_cols]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)

    print("PCA Results:")
    print(f"Explained Variance Ratio: {pca.explained_variance_ratio_}")
    print(f"Total Explained Variance: {np.sum(pca.explained_variance_ratio_):.4f}")

    pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])
    pca_df['player_id'] = df['player_id'].values


    return pca_df, pca, pca.explained_variance_ratio_



def calculate_shap_scores(model, df, feature_cols=None):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id', ]]

    X = df[feature_cols]

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)

    shap_df = pd.DataFrame(shap_values, columns=[f'{col}_shap' for col in feature_cols])
    shap_df['player_id'] = df['player_id'].values

    print("SHAP Values Summary:")
    print(shap_df.describe())

    return shap_df

def apply_rfe(df, target_col, estimator, n_features_to_select=5, feature_cols=None):
    if feature_cols is None:
        feature_cols = [col for col in df.columns if col not in ['player_id',  target_col]]

    X = df[feature_cols]
    y = df[target_col]

    rfe = RFE(estimator=estimator, n_features_to_select=n_features_to_select)
    rfe.fit(X, y)

    selected_features = [feature_cols[i] for i in range(len(feature_cols)) if rfe.support_[i]]

    print("RFE Results:")
    print(f"Selected Features ({n_features_to_select}): {selected_features}")
    print(f"Feature Ranking: {rfe.ranking_}")

    return selected_features, rfe

In [ ]:
    df = pd.read_csv("/data.csv")
    rf_model, rf_output_df, rf_importance = train_random_forest(df, target_col='fantasy_score_total_ema')
    print("\nRandom Forest Output DataFrame:\n", rf_output_df.head())

    xgb_model, xgb_output_df, xgb_importance = train_xgboost(df.head(1000), target_col='fantasy_score_total_ema')
    print("\nXGBoost Output DataFrame:\n", xgb_output_df.head())

    pca_df, pca, variance_ratio = apply_pca(df)
    print("\nPCA Transformed DataFrame:\n", pca_df.head())



    shap_df = calculate_shap_scores(rf_model, df)
    print("\nSHAP Values DataFrame:\n", shap_df.head())

    estimator = RandomForestRegressor(random_state=42)
    selected_features, rfe = apply_rfe(df, target_col='fantasy_score_total_ema', estimator=estimator)
    print("\nSelected Features by RFE:", selected_features)

Random Forest Regressor Results:
Mean Squared Error: 2.8713
R² Score: 0.9941

Feature Importance:
               Feature  Importance
7    wickets_taken_ema    0.559032
9      runs_scored_ema    0.369282
1    catches_taken_ema    0.019693
6    runs_conceded_ema    0.010885
11    fours_scored_ema    0.010460
5     balls_bowled_ema    0.005026
10     balls_faced_ema    0.005011
12    sixes_scored_ema    0.004553
0           Unnamed: 0    0.004052
8          maidens_ema    0.003695
2   run_out_direct_ema    0.003461
3    run_out_throw_ema    0.002896
4   stumpings_done_ema    0.001953

Random Forest Output DataFrame:
   player_id  fantasy_score_total_ema  predicted_value
0  00029c30                24.349301        25.652948
1  0019ca13                 1.000000         1.936885
2  0030a57d                12.529592        12.889129
3  00321fff                28.588857        28.433435
4  003d49e6                23.006045        23.932560
XGBoost Regressor Results:
Mean Squared Error: 0.0538


# Rooney's personal version

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [16]:
url = 'https://media.githubusercontent.com/media/pAppr69/Win-Factor_yashjeet/refs/heads/main/mw_pw_profiles.csv'
df = pd.read_csv(url)

In [17]:
def get_info(df):
  info_dict = {
      "Column_Name": df.columns,
      "Data_Type": df.dtypes,
      "NaN_Percentage": (df.isna().sum() / len(df) * 100).round(2)
  }
  return pd.DataFrame(info_dict)
get_info(df)

,Column_Name,Data_Type,NaN_Percentage
player_id,player_id,object,0.00
match_id,match_id,object,0.00
gender,gender,object,0.00
balls_per_over,balls_per_over,int64,0.00
start_date,start_date,object,0.00
series_name,series_name,object,0.47
match_type,match_type,object,0.00
name_x,name_x,object,0.00
runs_scored,runs_scored,int64,0.00
player_out,player_out,int64,0.00


In [18]:
df = df[['player_id','start_date','gender','series_name','catches_taken', 'run_out_direct', 'run_out_throw', 'stumpings_done', 'balls_bowled',
              'runs_conceded', 'wickets_taken', 'maidens', 'dot_balls_as_bowler', 'dot_balls_as_batsman','player_team','opposition_team',
              'runs_scored', 'balls_faced', 'fours_scored', 'sixes_scored','fantasy_score_bowling', 'fantasy_score_batting', 'fantasy_score_total']]

Inserting new columns: STRIKE RATE | Economy   

In [21]:
df = df.dropna()

## Boundaries
df['boundaries'] = df['sixes_scored']+df['fours_scored']

## fielding
df['fielding'] = df['run_out_direct'] + df['run_out_throw'] + df['stumpings_done'] + df['catches_taken']

df.drop(['sixes_scored', 'fours_scored', 'run_out_direct', 'run_out_throw', 'stumpings_done', 'catches_taken'], axis=1)


df['strike_rate'] = np.where(df.balls_faced != 0, ((df['runs_scored'] / df['balls_faced']) * 100), 0)

df['economy'] = np.where(df.balls_bowled != 0, ((df['runs_conceded'] / df['balls_bowled']) * 100), 0)

## players with 0 bowls bowled have an ideal economy, can worsen our model
## distintion between bowlers and batsmen is important

In [6]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['player_id','gender', 'series_name', 'player_team', 'opposition_team']

label_encoder = LabelEncoder()

for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [7]:
df['start_date'] = pd.to_datetime(df['start_date'])
df = df.sort_values(by='start_date')
df.reset_index(inplace=True)

In [8]:
df.drop(columns=['index'], inplace=True)

## Using EMA to make my features based on form of the player

In [9]:
df.columns

Index(['player_id', 'start_date', 'gender', 'series_name', 'catches_taken',
       'run_out_direct', 'run_out_throw', 'stumpings_done', 'balls_bowled',
       'runs_conceded', 'wickets_taken', 'maidens', 'dot_balls_as_bowler',
       'dot_balls_as_batsman', 'player_team', 'opposition_team', 'runs_scored',
       'balls_faced', 'fours_scored', 'sixes_scored', 'fantasy_score_bowling',
       'fantasy_score_batting', 'fantasy_score_total'],
      dtype='object')

In [10]:
def calc_ema(df, span, cols=['run_out_direct','run_out_throw', 'sixes_scored','fours_scored','runs_scored','balls_faced',
                             'stumpings_done','runs_conceded','balls_bowled','wickets_taken','maidens','dot_balls_as_bowler']):
  df_ema = df.copy()
  for col in cols:
    if col in df_ema.columns:
      df_ema[f'{col}_ema'] = df_ema.groupby('player_id')[col].transform(lambda x: x.ewm(span=span, adjust=False).mean())
      df_ema.drop(col,axis=1, inplace=True)
    else:
        print(f"Warning: Column '{col}' not found in DataFrame.")
  return df_ema

In [11]:
df_ema = calc_ema(df,5)

In [12]:
X = df_ema.drop(columns=['fantasy_score_batting','fantasy_score_bowling','fantasy_score_total'])
y = df['fantasy_score_total']

In [20]:
X

,player_id,start_date,gender,series_name,catches_taken,dot_balls_as_batsman,player_team,opposition_team,run_out_direct_ema,run_out_throw_ema,sixes_scored_ema,fours_scored_ema,runs_scored_ema,balls_faced_ema,stumpings_done_ema,runs_conceded_ema,balls_bowled_ema,wickets_taken_ema,maidens_ema,dot_balls_as_bowler_ema
0,9621,2001-12-19,1,209,0,4,92,130,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,5246,2001-12-19,1,209,0,3,130,92,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,3.000000,0.000000,5.100000e+01,9.700000e+01,0.000000e+00,3.000000e+00,7.000000e+01
2,6364,2001-12-19,1,209,0,21,130,92,0.000000e+00,0.000000,0.000000e+00,0.000000,8.000000,27.000000,0.000000,6.400000e+01,1.640000e+02,0.000000e+00,5.000000e+00,1.280000e+02
3,11340,2001-12-19,1,209,1,45,92,130,0.000000e+00,0.000000,1.000000e+00,5.000000,43.000000,64.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,551,2001-12-19,1,209,0,36,130,92,0.000000e+00,0.000000,0.000000e+00,2.000000,14.000000,42.000000,0.000000,8.000000e+01,1.770000e+02,1.000000e+00,7.000000e+00,1.350000e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414747,10239,2025-05-20,0,299,0,11,213,318,6.776728e-10,0.013233,3.455721e-03,1.035774,10.621673,12.158346,0.000000,1.781543e+00,2.137215e+00,4.565855e-02,2.373828e-04,1.168132e+00
414748,2060,2025-05-20,0,172,0,9,319,279,3.853673e-03,0.000000,0.000000e+00,1.624593,17.943663,29.389301,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
414749,24,2025-05-20,0,172,1,9,319,279,4.004038e-03,0.359346,2.287146e-09,1.649824,15.248251,15.557458,0.000000,4.678332e+01,5.054965e+01,1.126489e+00,3.664324e-01,2.565186e+01
414750,5742,2025-05-20,0,172,0,0,319,279,7.315958e-02,0.000000,2.926383e-02,0.083923,1.049039,1.526721,0.000000,3.713292e+01,4.320740e+01,5.723367e-01,6.766395e-04,2.734678e+01


In [13]:
def train_random_forest(X,y):
  from sklearn.ensemble import RandomForestRegressor
  from sklearn.compose import ColumnTransformer
  from sklearn.pipeline import Pipeline
  from sklearn.model_selection import train_test_split, cross_val_score
  from sklearn.metrics import mean_squared_error, r2_score
  from sklearn.preprocessing import StandardScaler

  # numeric_cols = X.select_dtypes(include=['number']).columns

  # numeric_cols=['run_out_direct','run_out_throw','sixes_scored','fours_scored','runs_scored','balls_faced',
  #                            'stumpings_done','runs_conceded','balls_bowled','wickets_taken','maidens','dot_balls_as_bowler']
  # for col in numeric_cols:
  #   cols = [f'{col}_ema']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

  # ## pipeline
  # preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), cols)])

  # model = Pipeline(steps=[('preprocessor', preprocessor),
  #                         ('regressor', RandomForestRegressor(n_estimators=50, random_state=1))])
  model = RandomForestRegressor(n_estimators=50, random_state=1)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)


  ## evaluating the model
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print("-- Random Forest Evaluation Scores--")

  # scores = cross_val_score(model, X, y, cv=5, scoring = 'r2')
  print(f"Mean Squared Error: {mse:.4f}")
  print(f"R² Score: {r2:.4f}")

  importances = model.feature_importances_

  print('- Feature Wise Importance -')
  for feature, importance in zip(feature_names, importances):
    print(f"{feature}: {importance:.4f}")

  return model

In [14]:
train_random_forest(X.drop('start_date', axis=1),y)

-- Random Forest Evaluation Scores--
Mean Squared Error: 484.5981
R² Score: 0.7427
- Feature Wise Importance -


NameError: name 'feature_names' is not defined

Interestingly... My model is performing better when I do **NOT** use EMA in columns, random forest performs better. (R^2 = 0.990).

In [23]:
def train_xgboost_with_gridsearch(X, y):
    from xgboost import XGBRegressor
    from sklearn.model_selection import GridSearchCV, train_test_split
    from sklearn.metrics import mean_squared_error, r2_score


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

    #base model
    model = XGBRegressor(random_state=2, n_jobs=-1)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("-- XGBoost Scores --")
    print()
    print(f"Test MSE: {mse:.4f}")
    print(f"Test R²: {r2:.4f}")

    return model

In [27]:
train_xgboost_with_gridsearch(X,y)

-- XGBoost Scores --

Test MSE: 16.6222
Test R²: 0.9911


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=2, ...)

In [ ]:
def train_xgboost_with_gridsearch_ema(df):
    X = df.drop(columns=['fantasy_score_batting','fantasy_score_bowling','fantasy_score_total'])
    y = df['fantasy_score_total']

    ema_cols=['run_out_direct','run_out_throw', 'sixes_scored','fours_scored','runs_scored','balls_faced',
                             'stumpings_done','runs_conceded','balls_bowled','wickets_taken','maidens','dot_balls_as_bowler']

    for col in ema_cols:
        X[f'{col}_ema'] = X.groupby('player_id')[col].transform(lambda x: x.ewm(span=1, adjust=False).mean())
        X.drop(col, axis=1, inplace=True)

    from xgboost import XGBRegressor
    from sklearn.model_selection import GridSearchCV, train_test_split
    from sklearn.metrics import mean_squared_error, r2_score


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

    #base model
    model = XGBRegressor(random_state=2, n_jobs=-1)
    param_grid = {
        'n_estimators': [100, 150],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [4, 6, 8],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha': [0, 0.1],
        'reg_lambda': [1.0]
    }


    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='r2',
        cv=5,
        verbose=1,
        n_jobs=-1
    )

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("-- XGBoost Scores --")
    print()
    print("Best Parameters:", grid_search.best_params_)
    print(f"Test MSE: {mse:.4f}")
    print(f"Test R²: {r2:.4f}")

    return best_model

In [ ]:
# prompt: apply pca on the

# Applying PCA on the features (X)
pca_df, pca, variance_ratio = apply_pca(X)
print("\nPCA Transformed DataFrame:\n", pca_df.head())